In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
test_df = pd.read_csv("../data/processed/test.csv")

with open("../models/user_item_matrix.pkl", "rb") as f:
    user_item_matrix = pickle.load(f)

with open("../models/item_similarity.pkl", "rb") as f:
    item_similarity_df = pickle.load(f)


In [3]:
def predict_rating(user_id, book_id, user_item_matrix, item_similarity_df, k=10):
    if user_id not in user_item_matrix.index or book_id not in user_item_matrix.columns:
        return None

    user_ratings = user_item_matrix.loc[user_id]
    rated_books = user_ratings[user_ratings > 0]

    if rated_books.empty:
        return None

    similarities = item_similarity_df[book_id]
    similarities = similarities[rated_books.index]

    top_k = similarities.sort_values(ascending=False).head(k)

    numerator = np.dot(top_k.values, rated_books[top_k.index].values)
    denominator = np.sum(np.abs(top_k.values))

    if denominator == 0:
        return None

    return numerator / denominator


In [4]:
test_sample = test_df.sample(5000, random_state=42)
test_sample.head()


,user_id,book_id,rating
45222,a1f443f0f3a3744406d62b6a15182dd9,38504,4
100397,2d72c3e13b22f740152672b7e9803f85,137110,3
63589,ff6d428e075e07702736c9d047b2c046,197853,5
118513,596a9769265798e2cbb077edba617901,315263,5
48796,83531f5a6c04a20b5dbb11372fb5979e,82346,2


In [5]:
y_true = []
y_pred = []

for _, row in test_sample.iterrows():
    user = row['user_id']
    book = row['book_id']
    true_rating = row['rating']

    if user in user_item_matrix.index and book in user_item_matrix.columns:
        pred = predict_rating(
    user,
    book,
    user_item_matrix,
    item_similarity_df
)

        if pred is not None:
            y_true.append(true_rating)
            y_pred.append(pred)


In [6]:
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

rmse, mae


(0.8246029810303539, 0.6208215229411517)